# Boston Crimes:
**Creating a database on incidents involving crime in the Boston area**

# Creating the database and the schema
Create a database named crime_db and a schema named crimes for storing the tables for containing the crime data.

The database crime_db does not exist yet so we connect to dq.

In [1]:
import psycopg2 as p
conn = p.connect(dbname='dq', user='dq')
conn.autocommit = True
cur = conn.cursor()
cur.execute('CREATE DATABASE crime_db')
conn.autocommit = False
conn.close()

conn = p.connect(dbname='crime_db', user='dq')
conn.autocommit = True
cur = conn.cursor()
cur.execute('CREATE SCHEMA crimes;')


# Obtaining the Column Names and Sample
Obtain the header row and assign it to a variable named col_headers and obtain the first data row and assign it to a variable named first_row.

In [2]:
import csv as c;

with open('boston.csv') as f:
    read_file=c.reader(f);
    col_headers = next(read_file)
    first_row = next(read_file)
    
print(col_headers)
print(first_row)
        
col_headers_dict = {};
for key in col_headers:
    col_headers_dict[key]=0;
        
    

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


# Creating a function for analyzing column values
Create a function get_col_values that given a CSV file name and a column index computes the set of all distinct values in that column.

Use the function on each column to evaluate which columns have a lot of different values. Columns with a limited set of possible values are good candidates for enumerated datatypes.

In [3]:
def get_col_value_set(csv_filename, col_index):
    with open(csv_filename) as f:
        next(f);
        read_file = c.reader(f);
        crime_desc_set = set();
        for row in read_file:
            if row[col_index] not in crime_desc_set:
                crime_desc_set.add(row[col_index]);
    
    return crime_desc_set



for i in range(len(col_headers)):
    csv_filename = 'boston.csv';
    col_index = i;
    
    col_values = get_col_value_set(csv_filename, col_index);
        
    key = col_headers[i];
    col_values_len = len(col_values);
    col_headers_dict[key] = col_values_len; 
    print(key+': '+str(col_headers_dict[key]))
    

    
    
    


incident_number: 298329
offense_code: 219
description: 239
date: 1177
day_of_the_week: 7
lat: 18177
long: 18177


# Analyzing the maximum length of the description column¶
Use the get_col_set function to compute the maximum description length to decide an appropriate length for that field.

In [4]:
csv_filename = 'boston.csv';
col_index = 2;

col_values = get_col_value_set(csv_filename, col_index);
col_value_max_len = 0;

for value in col_values:
    value_len = len(value);
    if value_len > col_value_max_len:
        col_value_max_len = value_len
        
print("col_value_max_len: ",col_value_max_len)

col_value_max_len:  58


# Creating the table
We have create an enumerated datatype named weekday for the day_of_the_week since there there only seven possible values.

For the incident_number we have decided to user the type INTEGER and set it as the primary key. The same datatype was also used to represent the offense_code.

Since the description has at most 58 character we decided to use the datatype VARCHAR(100) for representing it. This leave some margin while not being so big that we will waste a lot of memory.

The date was represented as the DATE datatype. Finally, for the latitude and longitude we used DECIMAL datatypes.

In [5]:
print('\n',col_headers)
print('\n',first_row)


 ['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']

 ['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


We will use the same names for the column headers.

The number of different values of each column was:

    incident_number 298329
    offense_code       219
    description        239
    date              1177
    day_of_the_week      7
    lat              18177
    long             18177
From the result of printing first_row we see that kind of data that we have are:

    integer numbers
    integer numbers
    string
    date
    string
    decimal number
    decimal number
Only column day_of_the_week has a small range of values so we will only create an enumerated datatype for this column. Column offense_code is also a good candidate since there is probably a limited set of possible offense codes.

We saw that the offense_code column has size at most 59. To be on the safe side we will limit the size of the description to 100 and use the VARCHAR(100) datatype.

The lat and long column see to need to hold quite a lot of precision so we will use the decimal type.

In [6]:
q1 = "CREATE TYPE weekday AS ENUM('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');"  

q2 = """CREATE TABLE crimes.boston_crimes (
    incident_number INTEGER PRIMARY KEY,
    offense_code INTEGER,
    description VARCHAR(100),
    date DATE,
    day_of_the_week weekday,
    lat decimal,
    long decimal
    );"""

cur.execute(q1);
cur.execute(q2);

# Load the data into the table
We used the copy_expert to load the data as it is very fast and very succinct to use.

In [7]:

with open('boston.csv','r') as f:
    q3="""COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;"""
    cur.copy_expert(q3,f)

  

    

q4 = """SELECT * FROM crimes.boston_crimes;"""
cur.execute(q4)
print(len(cur.fetchall()))

    

298329


# Revoke public privileges
We revoke all privileges of the public public group on the public schema to ensure that users will not inherit privileges on that schema such as the ability to create tables in the public schema.

We also need to revoke all privileges in the newly created schema. Doing this also makes it so that we do not need to revoke the privileges when we create users and groups because unless specified otherwise, privileges are not granted by default.

In [8]:
q5 = """REVOKE ALL ON SCHEMA public FROM public;"""
q6 = """REVOKE ALL ON DATABASE crime_db FROM public;"""
cur.execute(q5)
cur.execute(q6)

# Creating the read only group
We create a readonly group with NOLOGIN because it is a group and not a user. We grant the group the ability to connect to the crime_db and the ability to use the crimes schema.

Then we deal with tables privileges by granting SELECT. We also add an extra line compared with what was asked. This extra line changes the way that privileges are given by default to the readonly group on new table that are created on the crimes schema. As we mentioned, by default no privileges are given. However we change it so that by default any user in the readonly group can issue select commands.

In [9]:
q7 = """CREATE GROUP readonly NOLOGIN;"""
q9 = """GRANT CONNECT ON DATABASE crime_db TO readonly"""
q11= """GRANT USAGE ON SCHEMA crimes TO readonly"""
q13= """GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;"""

cur.execute(q7)
cur.execute(q9)
cur.execute(q11)
cur.execute(q13)


# Creating the read-write group
We create a readwrite group with NOLOGIN because it is a group and not a user. We grant the group the ability to connect to the crime_db and the ability to use the crimes schema.

Then we deal wit tables privileges by granting SELECT, INSERT, UPDATE and DELETE. As before we change the default privileges so that user in the readwrite group have these privileges if we ever create a new table on the crimes schema.

In [10]:
q8 = """CREATE GROUP readwrite NOLOGIN;"""
q10= """GRANT CONNECT ON DATABASE crime_db TO readwrite"""
q12= """GRANT USAGE ON SCHEMA crimes TO readwrite"""
q14= """GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;"""  

cur.execute(q8)
cur.execute(q10)
cur.execute(q12)
cur.execute(q14)

# Creating one user for each group
We create a user named data_analyst with password secret1 in the readonly group.

We create a user named data_scientist with password secret2 in the readwrite group.

In [11]:
q15= """CREATE USER data_analyst WITH PASSWORD 'secret1';"""
q16= """GRANT readonly TO data_analyst;"""
cur.execute(q15)
cur.execute(q16)

q17= """CREATE USER data_scientist WITH PASSWORD 'secret2';"""
q18= """GRANT readwrite TO data_scientist;"""
cur.execute(q17)
cur.execute(q18)



# Test the database setup
Test the database setup using SQL queries on the pg_roles table and information_schema.table_privileges.

In the pg_roles table we will check database related privileges and for that we will look at the following columns:

    rolname: The name of the user / group that the privilege refers to.
    rolsuper: Whether this user / group is a super user. It should be set to      False on every user / group that we have created.
    rolcreaterole: Whether user / group can create users, groups or roles. It should be False on every user / group that we have created.
    rolcreatedb: Whether user / group can create databases. It should be False on every user / group that we have created.
    rolcanlogin: Whether user / group can login. It should be True on the users and False on the groups that we have created.
    
In the information_schema.table_privileges we will check privileges related to SQL queries on tables. We will list the privileges of each group that we have created.

In [12]:
q19= """SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin
            FROM pg_roles
            WHERE rolname IN ('readonly','readwrite','data_analyst','data_scientist');
     """ 

q20= """SELECT grantee, privilege_type
            FROM information_schema.table_privileges
            WHERE grantee IN ('readonly','readwrite');

     """

cur.execute(q19)
pg_roles = cur.fetchall()
for row in pg_roles:
    print(row)
    
print()

cur.execute(q20)
privileges = cur.fetchall()
for row in privileges:
    print(row)


conn.close()



('readonly', False, False, False, False)
('readwrite', False, False, False, False)
('data_analyst', False, False, False, True)
('data_scientist', False, False, False, True)

('readonly', 'SELECT')
('readwrite', 'INSERT')
('readwrite', 'SELECT')
('readwrite', 'UPDATE')
('readwrite', 'DELETE')
